# HW4 – Statistics and Data Analysis
### Differential Gene Expression in Acute Myocardial Infraction 

<center> 
    Student IDs: 204818181, 329827190
    </center>

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats 

## Introduction

Gene expression describes the process in which genes that are coded in the DNA of living organisms are transcribed into mRNA. This is part of the bigger process in which genes are being copied (transcribed), processed, translated and modified into the final product, usually a protein. Gene expression profiling measures the levels at which mRNA molecules pertaining to the genes profiled are observed in a sample.
In this exercise, we will perform guided analysis, comparing expression profiles of circulating endothelial cells (CECs) in patients with acute myocardial infraction to CECs in healthy controls. A comparison of two sample classes. You will then select one more gene expression dataset and perform your own analysis there.


## Data

The data set was taken from:

1)	Dataset record in NCBI:
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE66360

2)	Published paper: Muse et al, Sci Rep 2017 https://www.nature.com/articles/s41598-017-12166-0

We extracted the data matrix and provide it as a separate csv attachment (link to download - https://drive.google.com/file/d/1-mu1J2wnlDIVNzoCXw20jJQ9r-h1jGrZ/view ). The csv file needs to be pre-processed before moving to the main analysis steps. Some information should be removed but make sure that you keep all information that is important for the analysis.
The paper describes a study that seeks to develop an expression-based signature that can detect AMI in patients in a non-invasive manner, by profiling CECs. 


In [2]:
# Skipped the first 60 rows of data as they are invalid
dataset = pd.read_csv("data.csv", skiprows=58, low_memory=False)

In [3]:
# Setting up data DataFrame
header = dataset.iloc[0]
data = dataset[1:] 
data.columns = header
type(data)
data

,Class,H,H,H,H,H,H,H,H,H,...,M,M,M,M,M,M,M,M,M,M
1,ID_REF,GSM1620819,GSM1620820,GSM1620821,GSM1620822,GSM1620823,GSM1620824,GSM1620825,GSM1620826,GSM1620827,...,GSM1620908,GSM1620909,GSM1620910,GSM1620911,GSM1620912,GSM1620913,GSM1620914,GSM1620915,GSM1620916,GSM1620917
2,1007_s_at,5.866377893,4.948736331,5.148384393,5.064418945,5.017832878,5.116153518,5.431328058,5.235270857,5.773528455,...,5.419481538,5.057716465,5.996493392,5.343132759,5.558892254,6.472517225,5.678815851,5.653286378,6.013841046,5.465333944
3,1053_at,8.22579012,7.525865369,7.764636529,7.198461482,7.831450258,7.203591859,7.694550756,7.760259212,8.279814404,...,7.226347747,7.105537863,5.354105386,8.271499725,5.96334574,8.261421952,6.925752665,7.918424183,7.442701377,9.225221352
4,117_at,6.17973231,6.628137025,5.859151477,5.974135101,6.793079158,6.43522914,6.320546126,6.48359047,6.387779205,...,6.789500767,7.441939912,7.026928573,6.003972814,7.474325713,6.367759272,6.835755831,7.577034915,7.147417202,6.48112813
5,121_at,6.179477974,6.58288903,6.602134766,6.545904723,5.911542321,6.28542026,6.562315839,6.345887555,6.27280582,...,6.513436777,6.317522639,7.413453376,6.809103167,5.949091368,6.440978114,6.955883278,6.499215058,6.587065112,6.897590966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54672,AFFX-ThrX-5_at,3.139333661,3.039961017,3.220109351,3.263781541,3.138524252,3.043078536,3.231107638,3.304715967,3.384011442,...,3.525011316,3.309030335,3.663399948,3.32400503,4.619261018,5.558780351,3.416999272,3.699181146,3.48527965,4.211921779
54673,AFFX-ThrX-M_at,2.7470148,2.63860588,2.495261011,2.544198973,2.506629527,2.817981885,2.525060286,2.91727791,2.643849254,...,2.930794418,2.745955351,3.165167607,2.899104156,4.066775612,5.429436708,2.834741433,3.24968301,2.71141329,3.346834702
54674,AFFX-TrpnX-3_at,2.651554479,2.643615067,2.626028059,2.504957719,2.609707404,2.521610278,2.840601479,2.509774164,2.696820467,...,2.788852359,2.725246769,3.143400613,2.76875475,3.546116866,4.487940292,2.782355764,3.056726837,2.833818355,3.06954169
54675,AFFX-TrpnX-5_at,3.413429017,3.399062751,3.539969696,3.396225335,3.43472012,3.516089622,3.514949337,3.475952299,3.495198562,...,3.701271488,3.456716807,4.127459216,3.668326993,4.889324389,4.254285925,3.882079933,3.649826789,3.523410023,4.108475085


## Analysis

### A.	High level description of the data and some pre-processing

1)	How many genes profiled? 

In [4]:
print('There are {} profiled genes'.format(data.shape[0] - 1))

There are 54675 profiled genes


2)	How many samples (subjects/patients) in total?

In [5]:
print('There are {} patients'.format(data.shape[1] - 1))

There are 99 patients


3)	How many samples in each class?

In [6]:
# We will transpose this data so it makes more sense to work with
transpose_temp = data.transpose()
transpose_temp = transpose_temp.reset_index()
transpose_header = transpose_temp.iloc[0]
transpose_data = transpose_temp[1:] 
transpose_data.columns = transpose_header
transpose_data

,Class,ID_REF,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
1,H,GSM1620819,5.866377893,8.22579012,6.17973231,6.179477974,2.792047952,9.290410779,6.998285145,5.348943925,...,13.25732501,13.26623454,14.57122985,14.27656808,4.504659469,3.139333661,2.7470148,2.651554479,3.413429017,3.140611771
2,H,GSM1620820,4.948736331,7.525865369,6.628137025,6.58288903,2.69571445,9.462275035,6.558214949,5.410884095,...,13.59386023,13.51701336,14.73883363,14.34123318,4.409225776,3.039961017,2.63860588,2.643615067,3.399062751,3.132691213
3,H,GSM1620821,5.148384393,7.764636529,5.859151477,6.602134766,2.580181122,9.116776316,6.851622539,5.254073031,...,13.37275868,13.41658291,14.62718054,14.31856805,4.310075612,3.220109351,2.495261011,2.626028059,3.539969696,3.698396097
4,H,GSM1620822,5.064418945,7.198461482,5.974135101,6.545904723,2.712123845,9.134584186,7.357827486,5.242139865,...,13.49022556,13.49482728,14.75412393,14.3632965,4.439310699,3.263781541,2.544198973,2.504957719,3.396225335,3.638611866
5,H,GSM1620823,5.017832878,7.831450258,6.793079158,5.911542321,2.723686912,9.582343511,7.283872601,5.219483996,...,13.23678545,13.33019244,14.54838931,14.26367675,4.562883241,3.138524252,2.506629527,2.609707404,3.43472012,3.577649067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,M,GSM1620913,6.472517225,8.261421952,6.367759272,6.440978114,3.73013912,7.355729877,6.206879166,6.217929861,...,6.601745953,5.94230757,7.290513508,8.988510881,5.567947288,5.558780351,5.429436708,4.487940292,4.254285925,5.375810973
96,M,GSM1620914,5.678815851,6.925752665,6.835755831,6.955883278,2.914867562,7.944294937,6.638364166,5.992343792,...,14.2710373,14.16370633,14.88450605,14.79250553,4.902195229,3.416999272,2.834741433,2.782355764,3.882079933,3.744138165
97,M,GSM1620915,5.653286378,7.918424183,7.577034915,6.499215058,3.013901455,8.563122679,7.379567509,5.766028609,...,13.89861618,13.68242475,14.88450605,14.78380144,5.239906199,3.699181146,3.24968301,3.056726837,3.649826789,4.021536308
98,M,GSM1620916,6.013841046,7.442701377,7.147417202,6.587065112,2.729319786,8.699221635,6.598349589,5.577456268,...,13.84700646,13.55159338,14.85806628,14.7781659,4.556134169,3.48527965,2.71141329,2.833818355,3.523410023,3.81256443


In [7]:
print('There {0} Hs and {1} Ms'.format(list(transpose_data["Class"].value_counts())[0],list(transpose_data["Class"].value_counts())[1]))

There 50 Hs and 49 Ms


4)	If there are missing values, then remove the entire row (gene) from the data matrix. How many rows left now?

In [8]:
transpose_data.dropna(how='any',axis=0,inplace=True)
print('There are {} genes'.format(transpose_data.shape[1] - 1))

There are 54676 genes


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### B.	WRS for differential expression (DE)

1)	Consider some gene, g. Under the null model (which assumes that for g there is no M vs H DE), what is the expected sum of ranks of g’s expression levels measured for samples labeled M?

$$E[RS(g)]= \frac{B(N+1)}{2} = \frac{B(g(M)+1)}{2} = \frac{49·100}{2} =2450$$

2)	Denote this sum of ranks by RS(g). What is the minimal value, m, that RS(g) can take?

We will take the sum of the first 49 ranks
$$ \sum^{49}_{i=1} RS(g) = \frac{n(n+1)}{2} =\frac{49(50)}{2} = 1225 $$

3)	Under the null model, what is the probability of RS(g) = m? 
(provide a formula for this and explain it)

In order to find the probability of RS(g) = m, we will consider the number of class premutation from 99 patients. 

That is $ {99 \choose 49} $ possible premutations. Then
$$ P[RS(g) = m] = \frac{1}{{99 \choose 49}} $$

4)	Under the null model, what is the probability of RS(g) = m+1? what is the probability of RS(g) = m+2? 
(provide formulas and explain them)

Under the null model, for $RS(g) = m + 1 $, we have
$$ P[RS(g) = m] = \frac{1}{{99 \choose 49}} $$
and from $RS(g) = m + 2 $, we have
$$ P[RS(g) = m] = \frac{2}{{99 \choose 49}} $$

5)	Draw a histogram of the values of RS(g) in the dataset. Here g ranges over all genes in the data (after the clean-up)

In [11]:
def rank_sum(gene):
    rank = gene.rank()
    print( ranks[transpose_data['Class'] == "M"].sum())

transpose_data['Class'] == "M"
for gene in transpose_data:
    rank_sum(gene)
    
# ranked_df = transpose_data.iloc[:,2:].apply(rank_sum, axis=0)
# ranked_df.hist(bins=30, density=True)
# plt.title("RS(g) over all genes")
# plt.ylabel("Density")
# plt.xlabel("RS(g)")

AttributeError: 'str' object has no attribute 'rank'

### C. Differential Expression
The purpose is to determine the statistical significance of differential expression (DE) observed for each gene in H vs M.
Evaluate the DE in both one-sided directions for every gene, using both Student t-test and WRS test. 
Report the number of genes overexpressed in M vs H at a p-value better (≤) than 0.05 and separately genes underexpressed in M vs H at a p-value of 0.05. For both directions use both a Student t-test and a WRS test.



### D.	Correlations
Select the 60 most significant genes from each one of the one-sided WRS DE lists you computed in 3c. Generate a set of 120 genes, D, which is the union of the above two sets.
Compute Spearman rho correlations in all pairs within D (120 choose 2 numbers). 

1)	What can you report about co-expression of genes in D (co-expression is inferred from the correlation of the expression levels of genes, across a set of samples)? Do we observe any significant co-expression? If so how many pairs, etc.

2)	What would have been advantages and disadvantages of computing co-expression for all genes in the study rather than only for genes in D? 


3)	Perform the above steps on the same set D, but restrict attention only to samples labeled M. What do you see now? Can you explain this?


### E.	Plots and Conclusions of the DE and correlation analysis

1)	Construct the DE overabundance plots (blue and green lines as shown in class) for M vs H overexpression (higher expression levels in M) using WRS and t-test using the results you had computed in Section 3c. 
State, for each comparison, the number of genes, k, at which we observe:

     a)	FDR = 0.1
     b)	FDR = 0.05
     c)	FDR = 0.001

If these events are not observed at any k, then make that statement.

2)	What can you say about the difference in results obtained in WRS vs those obtained by Student t-test?

3)	Select any 3 differentially expressed genes, from D (which was defined in 3d), and produce a graphical representation of their expression patterns that demonstrates the observed DE.

4)	Heatmap - Draw a heatmap representation of the expression values of the genes in D (from 3d), across the entire cohort (all samples). Order the genes and the samples to produce the maximal visual effect.
